In [1]:
import torch
import torch.nn as nn
import numpy as np

from model import FourierNet
from utils import ODESolver, select_available

device = torch.device('cpu')
# device = select_available()

In [2]:
features_in = 1
features_out = 1
hidden = [100, 100]

model = FourierNet(features_in, hidden, features_out)

$y'(x) = y(x); y(0) = 1$

$y(x) = e^{x}$

In [3]:
def rhs_function(x, y):
    return y

def sol(x):
    return torch.exp(x)

initial_conditions = [0, 1]

solver = ODESolver(model, rhs_function, initial_conditions, norm='L2', device=device)

In [4]:
domain = torch.linspace(0.0, 1.5, steps=100).unsqueeze(1)
solution = sol(domain)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10000

solver.train(domain, num_epochs, optimizer, solution=solution, atol=1e-5, save_gif=True)
solver.create_gif(gif_save_path='gif_test3')

Training Progress: 100%|█████████████| 10000/10000 [01:09<00:00, 143.87it/s, Train Loss: 0.01401004]


GIF saved at gif_test3/cas_100_100.gif


In [5]:
model.eval()
parameter_count = sum(p.numel() for p in model.parameters())

x_point = torch.tensor([1.0], requires_grad=True).to(device)
output = model(x_point).to(device)
output.backward()

exact_value = sol(x_point).item()
exact_derivative = sol(x_point).item()

value_error = abs(output.item() - exact_value)
derivative_error = abs(x_point.grad.item() - exact_derivative)

results = (
    f"Model parameter count: {parameter_count}\n"
    f"--- At x={x_point.item():.4f} ---\n"
    f"Prediction: {output.item():.4f}, Exact: {exact_value:.4f}, Abs Error: {value_error:.4f}\n"
    f"Derivative: {x_point.grad.item():.4f}, Exact: {exact_derivative:.4f}, Abs Error: {derivative_error:.4f}"
)

print(results)

Model parameter count: 20301
--- At x=1.0000 ---
Prediction: 2.6695, Exact: 2.7183, Abs Error: 0.0488
Derivative: 2.6675, Exact: 2.7183, Abs Error: 0.0507
